In [12]:
import gurobipy as gp
from gurobipy import GRB

# ───── 1. 데이터 정의 ─────
K = [0, 1]   # OD pair set
N = [1, 2, 3, 4]   # Node set
A = [(1,2), (1,4), (2,3), (4,3), (2,4)]   # Arc set (tuples like (i,j))

q = {0: 10, 1: 15}   # demand of k
o = {0: 1, 1: 1}   # origin of k
t = {0: 3, 1: 3}   # destination of k
S = {0: 3, 1: 4}   # scheduled arrival time
z = {0: 25, 1: 30}   # shortest path distance
C = {(1,2): 20, (1,4): 10, (2,3): 20, (2,4): 10, (4,3): 15}   # capacity per arc
L = {(1,2): 10, (1,4): 20, (2,3): 15, (2,4): 10, (4,3): 5}   # length per arc
B = {(1,2): 1, (1,4): 1, (2,3): 1, (2,4): 1, (4,3): 1}   # operational status of arcs
s = {0: 15, 1: 20}
tau = 20      # 20분 time step
v_avg = 60    # km/h → 1분에 1km → 30분이면 30km 가능
epsilon = 1
M = 1e5
w1, w2 = 1, 1

# ───── 2. 모델 생성 ─────
model = gp.Model("DelayAwareFlow")

# ───── 3. 변수 정의 ─────
f = model.addVars(K, A, name="f", lb=0)
u = model.addVars(K, name="u", lb=0)
h = model.addVars(K, vtype=GRB.BINARY, name="h")
theta = model.addVars(K, vtype=GRB.BINARY, name="theta")
delta = model.addVars(K, vtype=GRB.BINARY, name="delta")
a = model.addVars(K, name="a", lb=0)
d = model.addVars(K, name="d", lb=0)
g = model.addVars(K, name="g", lb=0)

# ───── 4. 목적함수 ─────
model.setObjective(
    w1 * gp.quicksum(delta[k] for k in K) +
    w2 * gp.quicksum(1 - h[k] for k in K),
    GRB.MINIMIZE
)

# ───── 5. 제약식 추가 ─────

# (1) Flow conservation
for k in K:
    for i in N:
        inflow = gp.quicksum(f[k, j, i] for j, ii in A if ii == i)
        outflow = gp.quicksum(f[k, i, j] for ii, j in A if ii == i)
        if i == o[k]:
            model.addConstr(outflow - inflow == q[k] - u[k], name=f"cons.1.origin_{k}_{i}")
        elif i == t[k]:
            model.addConstr(outflow - inflow == u[k] - q[k], name=f"cons.1.dest_{k}_{i}")
        else:
            model.addConstr(outflow - inflow == 0, name=f"cons.1.transit_{k}_{i}")

# (2) Capacity
for i, j in A:
    model.addConstr(
        gp.quicksum(f[k, i, j] for k in K) <= B[i, j] * C[i, j],
        name=f"cons.2.cap_{i}_{j}"
    )
for k in K:
    for i, j in A:
        model.addConstr(
            f[k, i, j] <= h[k] * s[k],
            name=f"cons.2.seat_{k}_{i}_{j}"
        )

# (3) Travel distance limit
for k in K:
    model.addConstr(
        gp.quicksum(L[i, j] * f[k, i, j] for i, j in A)
        <= (z[k] + v_avg * epsilon) * gp.quicksum(f[k, i, j] for i, j in A if i == o[k]),
        name=f"cons.3.travel_limit_{k}"
    )

# (4) Delay time constraints
for k in K:
    model.addConstr(g[k] >= q[k] * h[k] - M * (1 - theta[k]), name=f"cons.4.delay_g_lower_{k}")
    model.addConstr(g[k] + epsilon <= q[k] * h[k] + M * theta[k], name=f"cons.4.delay_g_upper_{k}")
    model.addConstr(theta[k] <= h[k], name=f"cons.4.theta_leq_h_{k}")
    model.addConstr(d[k] >= a[k] - S[k], name=f"cons.4.delay_def_{k}")
    model.addConstr(d[k] <= M * delta[k], name=f"cons.4.delay_d_upper_{k}")
    model.addConstr(d[k] >= epsilon * delta[k], name=f"cons.4.delay_d_lower_{k}")

# (5) Delay time calculation
for k in K:
    model.addConstr(
        gp.quicksum(L[i, j] * f[k, i, j] for i, j in A) <= v_avg * tau / 60,
        name=f"cons.5.time_limit_{k}"
    )

# ───── 6. Solve ─────
model.optimize()

# ───── 7. 결과 출력 ─────
if model.status == GRB.OPTIMAL:
    print(f"\n[✓] Optimal Objective Value: {model.ObjVal}\n")

    print("=== Flow (f[k, i, j]) ===")
    for k in K:
        for i, j in A:
            val = f[k, i, j].X
            if val > 1e-6:  # 출력 기준 설정 (0에 가까운 건 생략)
                print(f"f[{k}, {i}->{j}] = {val:.2f}")

    print("\n=== Service Variables ===")
    for k in K:
        print(f"OD {k}: u = {u[k].X:.2f}, h = {int(h[k].X)}, s = {s[k]}, theta = {int(theta[k].X)}")

    print("\n=== Delay Information ===")
    for k in K:
        print(f"OD {k}: a = {a[k].X:.2f}, d = {d[k].X:.2f}, delta = {int(delta[k].X)}, g = {g[k].X:.2f}")
else:
    print("[✗] No optimal solution found.")

# ───── 8. Infeasibility Analysis (if needed) ─────
if model.status == GRB.INFEASIBLE:
    print("\n[✗] Model is infeasible. Computing IIS (Irreducible Infeasible Set)...")
    model.computeIIS()
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Infeasible constraint: {c.ConstrName}")
else:
    print("\n[✓] No infeasibility detected.")



Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600XT 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 39 rows, 24 columns and 102 nonzeros
Model fingerprint: 0x84b6afef
Variable types: 18 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 39 rows and 24 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

[✓] Optimal Objective Value: 0.0

=== Flow (f[k, i, j]) ===

=== Service Variables ===
O

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

K = 2
T = 6

# Scheduled and actual arrival matrices
S_kt = np.array([
    [1, 2, 3, 3, 3, 3],
    [1, 4, 4, 3, 3, 3]
])
A_kt = np.array([
    [1, 1, 5, 6, 7, 3],
    [1, 4, 4, 4, 3, 3]
])
N = list(range(1, 8))

model = gp.Model("Delay_Cancel_Model")

# Time lookup tables
s_t, a_t = {}, {}
for k in range(K):
    for t in range(T):
        s_node = S_kt[k, t]
        a_node = A_kt[k, t]
        if (k, s_node) not in s_t: s_t[(k, s_node)] = t
        if (k, a_node) not in a_t: a_t[(k, a_node)] = t

d, c, v = {}, {}, {}

for k in range(K):
    for n in N:
        d[k, n] = model.addVar(lb=0, name=f"d_{k}_{n}")
        c[k, n] = model.addVar(vtype=GRB.BINARY, name=f"c_{k}_{n}")
        v[k, n] = model.addVar(vtype=GRB.BINARY, name=f"v_{k}_{n}")

        # Visitation constraint (corrected)
        if (k, n) in a_t:
            model.addConstr(v[k, n] == 1)
        else:
            model.addConstr(v[k, n] == 0)

        # Cancellation constraint
        if (k, n) in s_t:
            if (k, n) in a_t:
                model.addConstr(c[k, n] == 0)
            else:
                model.addConstr(c[k, n] == 1)
        else:
            model.addConstr(c[k, n] == 0)

        # Delay constraint
        if (k, n) in s_t and (k, n) in a_t:
            model.addConstr(d[k, n] >= a_t[(k, n)] - s_t[(k, n)])
        else:
            model.addConstr(d[k, n] == 0)

model.setObjective(
    gp.quicksum(c[k, n] + d[k, n] for k in range(K) for n in N),
    GRB.MINIMIZE
)

model.optimize()

for k in range(K):
    print(f"\n[OD {k}]")
    for n in N:
        print(f"Node {n}: visited={int(v[k, n].X)}, cancel={int(c[k, n].X)}, delay={d[k, n].X:.1f}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600XT 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 42 rows, 42 columns and 42 nonzeros
Model fingerprint: 0x05cd7f5e
Variable types: 14 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 5.0000000
Presolve removed 42 rows and 42 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%

[OD 0]
Node 1: visited=1, cancel=0, delay

: 